In [7]:
#policy model
import requests

url = "http://127.0.0.1:30020/generate"  
text = "What is 2+2? 4"

response = requests.post(url, json={"text": text})

if response.status_code == 200:
    print(response.json()) 
else:
    print(f"Error: {response.status_code}")


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=30020): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6a640992a0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
#reward model
import requests

url = "http://127.0.0.1:30010/generate"  
text = "What is 2+2? 4"

response = requests.post(url, json={"text": text})

if response.status_code == 200:
    print(response.json()) 
else:
    print(f"Error: {response.status_code}")

{'text': ' труokat became methods Because msцияdelayത caused deel}</ accordingadmin�рома', 'meta_info': {'prompt_tokens': 10, 'completion_tokens': 16, 'completion_tokens_wo_jump_forward': 16, 'id': 'c4ccf0cdc158429d9fa8d993c7f9ee7d'}, 'scores': [], 'forward_only': False}


In [ ]:
# import sys
# sys.path.append('/data/irving/llm-reasoners/examples/ScalableTreeSearch/sglang/python')
import argparse
import sglang as sgl
from sglang import RuntimeEndpoint, function, gen
from sglang.test.test_utils import (
    add_common_sglang_args_and_parse,
    select_sglang_backend,
)
import time

In [4]:
import random
import re
from typing import Literal

import blobfile as bf
import pandas

import common
from common import ANSWER_PATTERN, HTML_JINJA, check_equality
from eval_types import Eval, EvalResult, SamplerBase, SingleEvalResult

split = "my_math_500_test"
df = pandas.read_csv(
    bf.BlobFile(f"{split}.csv")
)
examples = [row.to_dict() for _, row in df.iterrows()]

In [5]:
args = argparse.Namespace(
    # data_path="sglang/benchmark/tree_of_thought_v0/test.jsonl",
    # num_questions=2,
    port=10086,
    parallel=16,
    backend='srt',
    host="http://127.0.0.1",
    result_file="results.txt"
)

# q = """If $x^3$ is a positive factor of $10!,$ how many possible integer values of $x$ are there?  (Reminder: For a positive integer $n$, the expression $n!$ stands for the product of the integers from 1 up to (and including) $n$.)"""

arguments = [{"question": d['Question']} for d in examples[:100]]

NameError: name 'argparse' is not defined

In [6]:
reward_host = RuntimeEndpoint("http://127.0.0.1:30010")
policy_host = RuntimeEndpoint("http://127.0.0.1:30020")

NameError: name 'RuntimeEndpoint' is not defined

In [7]:
#select width state with highest score
def select_top_k_state(state_list, state_scores, width):
    state_score_pairs = list(zip(state_list, state_scores))
    
    sorted_state_score_pairs = sorted(state_score_pairs, key=lambda pair: pair[1], reverse=True)

    selected_states = [pair[0] for pair in sorted_state_score_pairs[:width]]
    
    return selected_states

#obtain reward_model score
def obtain_score(state):
    if state.scores() == [] or state.scores == None:
        return 0
    else:
        return state.scores()[-1]

In [1]:
import sglang as sgl
from pprint import pprint
max_steps = 30

BEAM_SIZE = 4
BEAM_WIDTH = 2
assert BEAM_SIZE % BEAM_WIDTH == 0

@sgl.function
def beam_search(s, question):
    s += sgl.user(
        f"""Solve the following math problem step by step. Steps should be separated with two new lines. The last line of your response should be of the form Answer: $ANSWER (without quotes) where $ANSWER is the answer to the problem.
        
{question}

Remember to separate steps with two new lines, and finally put your answer on its own line after "Answer:", and you do not need to use a \\boxed command."""
    )

    s += sgl.assistant_begin()
    forks = s.fork(BEAM_SIZE)
    for fork in forks:
        fork.set_score_backend(reward_host)
    forks += sgl.gen("step", max_tokens=256, stop=["\u043a\u0438"], temperature=0.5)
    forks += sgl.gen("score", max_tokens=0, forward_only=True, logits_require_id=8094)

    step = s.text().split("\n\n")
    
    cur_states = list(forks)
    cur_scores = [obtain_score(fork) for fork in forks]
    
    answer_states = []

    for _ in range(max_steps):
        
        cur_beam_width = min(BEAM_WIDTH, len(cur_states))
        
        cur_states = select_top_k_state(cur_states, cur_scores, cur_beam_width)
        # expand to BEAM_SIZE states
        new_states = []
        new_scores = []

        for state in cur_states:

            # print("--B--")
            latest_step = state.text().split("\u043a\u0438")[-1].strip()
            print(latest_step)
            if "Answer:" in latest_step:# or "The answer is:" in latest_step:
                answer_states.append(state)
                continue
            
            # print("--C--")
            forked_states = state.fork((BEAM_SIZE - 1) // cur_beam_width + 1)
            for fork_state in forked_states:
                fork_state.set_score_backend(reward_host)
            forked_states +=  "\n\n" + sgl.gen("step", max_tokens=256, stop=["\u043a\u0438"], temperature=0.5)
            forked_states += sgl.gen("score", max_tokens=0, forward_only=True, logits_require_id=8094)
            
            new_states.extend(forked_states)
            new_scores.extend([obtain_score(fork) for fork in forked_states])
            
        
        # print("--D--")
        cur_states = new_states
        cur_scores = new_scores
        
        #pprint(list(cur_states))

        if len(answer_states) > 0:
            break
            
        
    return answer_states


In [2]:
bs_states = beam_search.run_batch(
    arguments[0:5],
    temperature=0,
    backend=policy_host,
    num_threads=args.parallel,
    progress_bar=True,
)

NameError: name 'arguments' is not defined

In [32]:
from dart_math.eval import EvaluatorMath
math_evaluator = EvaluatorMath()

/data/irving/anaconda3/envs/sglang/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/irving/anaconda3/envs/sglang/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-11-25 19:36:03,098	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [33]:
scores = 0

for i, states in enumerate(bs_states):
    if len(states.ret_value) == 0:
        continue

    response_text = states.ret_value[0].text()
    match = re.search(ANSWER_PATTERN, response_text.split("\n")[-1])
    extracted_answer = match.group(1) if match else None
    # score = 0 if extracted_answer is None else \
    #     float(check_equality(self.equality_checker, row["Answer"], extracted_answer))
    answer = examples[i]["Answer"]
    score = 0 if extracted_answer is None else math_evaluator.eq(answer, extracted_answer)
    
    print(f"{extracted_answer} || {answer} || {score}")
    scores += score

print(scores/len(bs_states))

126The answer is: 126 ки || \left( 3, \frac{\pi}{2} \right) || False
0.0


In [34]:
examples[0]['Answer']

'\\left( 3, \\frac{\\pi}{2} \\right)'

In [12]:
bs_states[0].ret_value

[ProgramState(USER:Solve the following math problem step by step. Steps should be separated with two new lines. The last line of your response should be of the form Answer: $ANSWER (without quotes) where $ANSWER is the answer to the problem.
         
 Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
 
 Remember to separate steps with two new lines, and finally put your answer on its own line after "Answer:", and you do not need to use a \boxed command.
 ASSISTANT:Step 1: To convert from rectangular coordinates to polar coordinates, we use the formulas $r = \sqrt{x^2 + y^2}$ and $\theta = \arctan\left(\frac{y}{x}\right)$. ),
 ProgramState(USER:Solve the following math problem step by step. Steps should be separated with two new lines. The last line of your response should be of the form Answer: $ANSWER (without quotes) where $ANSWER is the answer to the problem.
         
 C

5